In [1]:
import os
import pandas as pd
import pyspark
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, lit
from pyspark.sql.types import * 
import pandas as pd
import pyspark.sql.functions as sf

In [2]:
os.getcwd()

'/opt/workspace'

In [3]:
from helper import spark_helper

In [4]:
spark = spark_helper.get_spark_context("Class4")

22/11/20 02:32:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
path = '/opt/workspace/data/'
file_name = '20220401.json'
date = '2022-04-01'

In [6]:
df = spark.read.json(path+file_name)
df = df.select('_source.AppName','_source.Contract','_source.Mac','_source.TotalDuration')
df = df.withColumn('Date',lit(date))
df = df.withColumn('Type',
                when((col("AppName") == 'CHANNEL') | (col("AppName") =='DSHD')| (col("AppName") =='KPLUS')| (col("AppName") =='KPlus'), "Truyền Hình")
        .when((col("AppName") == 'VOD') | (col("AppName") =='FIMS_RES')| (col("AppName") =='BHD_RES')| 
                (col("AppName") =='VOD_RES')| (col("AppName") =='FIMS')| (col("AppName") =='BHD')| (col("AppName") =='DANET'), "Phim Truyện")
        .when((col("AppName") == 'RELAX'), "Giải Trí")
        .when((col("AppName") == 'CHILD'), "Thiếu Nhi")
        .when((col("AppName") == 'SPORT'), "Thể Thao")
        .otherwise("Error"))
df = df.select('Contract','Type','TotalDuration','Date')
# df = df.groupBy('Contract','Type','Date').agg({'TotalDuration':'sum'}).withColumnRenamed('sum(TotalDuration)','TotalDuration')

df.show()

+---------+-----------+-------------+----------+
| Contract|       Type|TotalDuration|      Date|
+---------+-----------+-------------+----------+
|HNH579912|Truyền Hình|          254|2022-04-01|
|HUFD40665|Truyền Hình|         1457|2022-04-01|
|HNH572635|Truyền Hình|         2318|2022-04-01|
|HND141717|Truyền Hình|         1452|2022-04-01|
|HNH743103|Truyền Hình|          251|2022-04-01|
|HNH893773|Truyền Hình|          924|2022-04-01|
|HND083642|Truyền Hình|         1444|2022-04-01|
|DNFD74404|Truyền Hình|          691|2022-04-01|
|DTFD21200|Truyền Hình|         1436|2022-04-01|
|LDFD05747|Truyền Hình|         1434|2022-04-01|
|HNH063566|Truyền Hình|          687|2022-04-01|
|HNH866786|Truyền Hình|          248|2022-04-01|
|NBAAA1128|Truyền Hình|          247|2022-04-01|
|HNH960439|Truyền Hình|          683|2022-04-01|
|HNJ035736|Truyền Hình|          246|2022-04-01|
|NTFD93673|Truyền Hình|         2288|2022-04-01|
|HNJ063267|Truyền Hình|         2282|2022-04-01|
|HNH790383|Truyền Hì

In [7]:
df = df.cache()

In [8]:
df.select('Contract','Type','TotalDuration','Date').show()

+---------+-----------+-------------+----------+
| Contract|       Type|TotalDuration|      Date|
+---------+-----------+-------------+----------+
|HNH579912|Truyền Hình|          254|2022-04-01|
|HUFD40665|Truyền Hình|         1457|2022-04-01|
|HNH572635|Truyền Hình|         2318|2022-04-01|
|HND141717|Truyền Hình|         1452|2022-04-01|
|HNH743103|Truyền Hình|          251|2022-04-01|
|HNH893773|Truyền Hình|          924|2022-04-01|
|HND083642|Truyền Hình|         1444|2022-04-01|
|DNFD74404|Truyền Hình|          691|2022-04-01|
|DTFD21200|Truyền Hình|         1436|2022-04-01|
|LDFD05747|Truyền Hình|         1434|2022-04-01|
|HNH063566|Truyền Hình|          687|2022-04-01|
|HNH866786|Truyền Hình|          248|2022-04-01|
|NBAAA1128|Truyền Hình|          247|2022-04-01|
|HNH960439|Truyền Hình|          683|2022-04-01|
|HNJ035736|Truyền Hình|          246|2022-04-01|
|NTFD93673|Truyền Hình|         2288|2022-04-01|
|HNJ063267|Truyền Hình|         2282|2022-04-01|
|HNH790383|Truyền Hì

In [9]:
df.select('Contract').distinct().count()

1360622

In [10]:
daily_statistic = df.groupBy('Date','Type').agg({'TotalDuration':'sum','Contract':'count'}).withColumnRenamed('sum(TotalDuration)','TotalDuration').withColumnRenamed('count(Contract)','TotalUsers')

In [11]:
daily_statistic.show()

+----------+-----------+----------+-------------+
|      Date|       Type|TotalUsers|TotalDuration|
+----------+-----------+----------+-------------+
|2022-04-01|Phim Truyện|     89040|    627150472|
|2022-04-01|Truyền Hình|   1523013|  25210339682|
|2022-04-01|   Giải Trí|      6445|     20907906|
|2022-04-01|  Thiếu Nhi|     33723|    178570178|
|2022-04-01|   Thể Thao|      2298|      1166238|
+----------+-----------+----------+-------------+



In [12]:
daily_statistic.withColumn('TotalDuration',col('TotalDuration')/3600).show()

+----------+-----------+----------+------------------+
|      Date|       Type|TotalUsers|     TotalDuration|
+----------+-----------+----------+------------------+
|2022-04-01|Phim Truyện|     89040|174208.46444444446|
|2022-04-01|Truyền Hình|   1523013| 7002872.133888889|
|2022-04-01|   Giải Trí|      6445| 5807.751666666667|
|2022-04-01|  Thiếu Nhi|     33723| 49602.82722222222|
|2022-04-01|   Thể Thao|      2298|           323.955|
+----------+-----------+----------+------------------+



In [13]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7fb08c44eb80>>

22/11/20 02:34:41 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/spark-f408398c-0856-410b-aeb3-8ed05f955740/pyspark-8b815098-9924-4797-bbdb-61de6ae7b13a. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/spark-f408398c-0856-410b-aeb3-8ed05f955740/pyspark-8b815098-9924-4797-bbdb-61de6ae7b13a
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:171)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:110)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:91)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1141)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4(ShutdownHookManager.scala:65)
	at org.apache.spark.util.ShutdownHookManager$.$anonfun$new$4$adapted(ShutdownHookManager.scala:62)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreac